# ADK Response Structure Debug and Analysis

This notebook will help us debug and understand the ADK response structure to fix the Streamlit app parsing issues.

## 1. Import Required Libraries

In [ ]:
import json
import re
import requests
from typing import Dict, List, Optional
from pprint import pprint

print("Libraries imported successfully!")

: 

## 2. Sample ADK Response Data

Let's use the actual response you're getting to debug the parsing logic:

In [ ]:
# Sample response structure from the user's issue
sample_response = [
    {
        'content': {
            'parts': [{
                'functionCall': {
                    'id': 'adk-d087df55-0403-47bd-93fd-cda384dad1a9',
                    'args': {'question': 'Write an educational story for kids about the importance of recycling'},
                    'name': 'call_content_gen_agent'
                }
            }],
            'role': 'model'
        },
        'usageMetadata': {'candidatesTokenCount': 19, 'promptTokenCount': 726, 'totalTokenCount': 745},
        'author': 'sahayak_agent',
        'id': '5df0b1ba-491b-443d-aa20-7b7f524ce26f'
    },
    {
        'content': {
            'parts': [{
                'functionResponse': {
                    'id': 'adk-d087df55-0403-47bd-93fd-cda384dad1a9',
                    'name': 'call_content_gen_agent',
                    'response': {
                        'result': '```\nTitle: The Adventures of Riya and the Recycling Robot\n\nStory:\nOnce upon a time, in a small village nestled among green fields, lived a bright young girl named Riya. Riya loved her village, but she noticed that sometimes people threw waste carelessly. One day, while exploring near the old schoolhouse, she stumbled upon a strange, boxy robot with big, friendly eyes.\n\n"Hello!" beeped the robot. "I am RecycleBot, and I\'m here to help clean up and teach about recycling!"\n\nRiya\'s eyes widened. "Recycling? What\'s that?"\n\nRecycleBot explained, "Recycling is when we take old things, like paper, plastic, and glass, and turn them into new things! Instead of throwing them away, we give them a new life."\n\nTogether, Riya and RecycleBot went around the village. They showed people how to separate their waste into different bins: blue for paper, green for glass, and yellow for plastic. At first, some people were hesitant. "It\'s too much work!" they grumbled.\n\nBut Riya and RecycleBot didn\'t give up. They explained that recycling helps keep the village clean, saves resources, and protects the animals. They showed them how old newspapers could become new notebooks, and plastic bottles could become park benches.\n\nSoon, the whole village joined in. Children collected paper from their homes, and adults sorted plastic after market day. The village became cleaner and greener.\n\nOne sunny morning, RecycleBot announced, "Riya, because of your hard work, the village has become a model for recycling! The river is cleaner, the fields are greener, and everyone understands the importance of taking care of our planet!"\n\nRiya smiled. She realized that even small actions, like recycling, could make a big difference.\n\nTakeaway:\nRecycling is important because it keeps our environment clean, saves resources, and protects our planet for future generations. Every small action counts!\n```\n'
                    }
                }
            }],
            'role': 'user'
        },
        'author': 'sahayak_agent',
        'id': 'de802afb-ce06-4dac-b116-f0b25c7184f1'
    },
    {
        'content': {
            'parts': [{
                'text': '**Result:**\n```\nTitle: The Adventures of Riya and the Recycling Robot\n\nStory:\nOnce upon a time, in a small village nestled among green fields, lived a bright young girl named Riya. Riya loved her village, but she noticed that sometimes people threw waste carelessly. One day, while exploring near the old schoolhouse, she stumbled upon a strange, boxy robot with big, friendly eyes.\n\n"Hello!" beeped the robot. "I am RecycleBot, and I\'m here to help clean up and teach about recycling!"\n\nRiya\'s eyes widened. "Recycling? What\'s that?"\n\nRecycleBot explained, "Recycling is when we take old things, like paper, plastic, and glass, and turn them into new things! Instead of throwing them away, we give them a new life."\n\nTogether, Riya and RecycleBot went around the village. They showed people how to separate their waste into different bins: blue for paper, green for glass, and yellow for plastic. At first, some people were hesitant. "It\'s too much work!" they grumbled.\n\nBut Riya and RecycleBot didn\'t give up. They explained that recycling helps keep the village clean, saves resources, and protects the animals. They showed them how old newspapers could become new notebooks, and plastic bottles could become park benches.\n\nSoon, the whole village joined in. Children collected paper from their homes, and adults sorted plastic after market day. The village became cleaner and greener.\n\nOne sunny morning, RecycleBot announced, "Riya, because of your hard work, the village has become a model for recycling! The river is cleaner, the fields are greener, and everyone understands the importance of taking care of our planet!"\n\nRiya smiled. She realized that even small actions, like recycling, could make a big difference.\n\nTakeaway:\nRecycling is important because it keeps our environment clean, saves resources, and protects our planet for future generations. Every small action counts!\n```\n\n**Explanation:**\nI invoked the `ContentGenerationAgent` to generate an educational story for kids about the importance of recycling. The agent returned a story about a girl named Riya and a robot named RecycleBot who teach a village about recycling.\n'
            }],
            'role': 'model'
        },
        'author': 'sahayak_agent',
        'id': '6d7ada1d-922f-432c-aa52-52f9c82c6ae5'
    }
]

print(f"Sample response has {len(sample_response)} items")
for i, item in enumerate(sample_response):
    print(f"Item {i}: {item.get('content', {}).get('parts', [{}])[0].keys()}")

## 3. Current Parsing Logic (From Streamlit App)

Let's implement the current parsing logic and see what it extracts:

In [ ]:
def current_format_adk_response(response):
    """Current format_adk_response logic from the Streamlit app"""
    # Handle different response structures
    if isinstance(response, dict):
        if 'response' in response:
            response_data = response['response']
        else:
            response_data = response
    else:
        response_data = response
    
    # Handle direct list response (new format)
    if isinstance(response_data, list):
        response_list = response_data
    elif 'candidates' in response_data:
        response_list = response_data['candidates']
    else:
        response_list = [response_data]
    
    print(f"Processing {len(response_list)} response items")
    
    # Look for text content first (usually contains the final formatted response)
    final_result = ""
    for i, item in enumerate(response_list):
        print(f"\n--- Processing item {i} ---")
        if 'content' in item and 'parts' in item['content']:
            for j, part in enumerate(item['content']['parts']):
                print(f"Part {j}: {list(part.keys())}")
                if 'text' in part:
                    text_content = part['text']
                    print(f"Found text content (length: {len(text_content)})")
                    if isinstance(text_content, str) and text_content.strip():
                        # Check if this text contains "**Result:**" - this is usually the final response
                        if "**Result:**" in text_content or "**Explanation:**" in text_content:
                            print("✅ Found text with Result/Explanation markers!")
                            final_result = text_content
                            break
            if final_result:
                break
    
    # If no text content found, look for function responses with results
    if not final_result:
        print("\n--- Looking for function responses ---")
        for i, item in enumerate(response_list):
            if 'content' in item and 'parts' in item['content']:
                for j, part in enumerate(item['content']['parts']):
                    if 'functionResponse' in part:
                        func_resp = part['functionResponse']
                        print(f"Found function response: {func_resp.get('name', 'unknown')}")
                        if 'response' in func_resp and 'result' in func_resp['response']:
                            result = func_resp['response']['result']
                            print(f"Found function result (length: {len(result)})")
                            if isinstance(result, str) and result.strip():
                                final_result = result
                                break
                if final_result:
                    break
    
    return final_result

# Test the current logic
result = current_format_adk_response(sample_response)
print(f"\n=== FINAL RESULT ===")
print(f"Length: {len(result)}")
print(f"Preview (first 200 chars): {result[:200]}...")

## 4. Text Cleaning Function

Now let's test the text cleaning function on the extracted result:

In [ ]:
def clean_response_text(text):
    """Clean up the response text by removing markdown artifacts while keeping explanations"""
    if not isinstance(text, str):
        return str(text)
    
    print(f"BEFORE CLEANING (length: {len(text)}):")
    print(f"First 200 chars: {text[:200]}")
    print("---")
    
    # Remove common markdown wrappers
    text = text.strip()
    
    # Remove **Result:** prefix
    if text.startswith('**Result:**'):
        text = text.replace('**Result:**', '').strip()
        print("✅ Removed **Result:** prefix")
    
    # Remove markdown code blocks (``` at start and end)
    if text.startswith('```') and text.endswith('```'):
        text = text[3:-3].strip()
        print("✅ Removed surrounding code blocks")
    
    # Remove any remaining ``` at the beginning or end (multiple passes)
    original_length = len(text)
    while text.startswith('```'):
        text = text[3:].strip()
    while text.endswith('```'):
        text = text[:-3].strip()
    
    if len(text) != original_length:
        print("✅ Removed additional code block markers")
    
    # Process line by line but keep everything including explanations
    lines = text.split('\n')
    final_lines = []
    
    for line in lines:
        # Keep all lines - don't filter out explanations anymore
        final_lines.append(line)
    
    # Join back and clean up extra whitespace
    cleaned_text = '\n'.join(final_lines).strip()
    
    # Remove multiple consecutive newlines
    cleaned_text = re.sub(r'\n\s*\n\s*\n', '\n\n', cleaned_text)
    
    # Final cleanup for any remaining ``` scattered in the text
    cleaned_text = re.sub(r'```\s*$', '', cleaned_text).strip()
    cleaned_text = re.sub(r'```', '', cleaned_text).strip()
    
    print(f"\nAFTER CLEANING (length: {len(cleaned_text)}):")
    print(f"First 200 chars: {cleaned_text[:200]}")
    
    return cleaned_text

# Test the cleaning function
if result:
    cleaned = clean_response_text(result)
    print(f"\n=== FINAL CLEANED RESULT ===")
    print(cleaned)
else:
    print("No result to clean!")

## 5. Improved Parsing Logic

Let's create a better parsing function that handles all cases properly:

In [ ]:
def improved_format_adk_response(response):
    """Improved format_adk_response logic that handles the structure better"""
    # Handle different response structures
    if isinstance(response, dict):
        if 'response' in response:
            response_data = response['response']
        else:
            response_data = response
    else:
        response_data = response
    
    # Handle direct list response (new format)
    if isinstance(response_data, list):
        response_list = response_data
    elif 'candidates' in response_data:
        response_list = response_data['candidates']
    else:
        response_list = [response_data]
    
    print(f"Processing {len(response_list)} response items")
    
    # Strategy: Look for the LAST text content that contains complete information
    # The ADK seems to build up the response progressively
    best_text_result = ""
    best_function_result = ""
    
    for i, item in enumerate(response_list):
        print(f"\n--- Processing item {i} ---")
        if 'content' in item and 'parts' in item['content']:
            for j, part in enumerate(item['content']['parts']):
                print(f"Part {j}: {list(part.keys())}")
                
                # Check for text content
                if 'text' in part:
                    text_content = part['text']
                    print(f"Found text content (length: {len(text_content)})")
                    if isinstance(text_content, str) and text_content.strip():
                        # Always update - we want the LAST/BEST text content
                        if "**Result:**" in text_content and "**Explanation:**" in text_content:
                            print("✅ Found complete text with both Result and Explanation!")
                            best_text_result = text_content
                        elif len(text_content) > len(best_text_result):
                            print("📝 Found longer text content")
                            best_text_result = text_content
                
                # Check for function response
                elif 'functionResponse' in part:
                    func_resp = part['functionResponse']
                    print(f"Found function response: {func_resp.get('name', 'unknown')}")
                    if 'response' in func_resp and 'result' in func_resp['response']:
                        result = func_resp['response']['result']
                        print(f"Found function result (length: {len(result)})")
                        if isinstance(result, str) and result.strip():
                            best_function_result = result
    
    # Prioritize: complete text result > function result > any text result
    if best_text_result and ("**Result:**" in best_text_result and "**Explanation:**" in best_text_result):
        print("🎯 Using complete text result with both parts")
        return best_text_result
    elif best_function_result:
        print("🔧 Using function result")
        return best_function_result
    elif best_text_result:
        print("📄 Using best available text result")
        return best_text_result
    else:
        print("❌ No suitable content found")
        return "Response received from agent (no readable content found)"

# Test the improved logic
print("=== TESTING IMPROVED LOGIC ===")
improved_result = improved_format_adk_response(sample_response)
print(f"\n=== IMPROVED RESULT ===")
print(f"Length: {len(improved_result)}")
print(f"Preview (first 300 chars): {improved_result[:300]}...")

# Clean it
if improved_result:
    print("\n=== CLEANING IMPROVED RESULT ===")
    final_cleaned = clean_response_text(improved_result)
    print(f"\n=== FINAL RESULT FOR STREAMLIT ===")
    print(final_cleaned[:500] + "..." if len(final_cleaned) > 500 else final_cleaned)